# Setup


In [2]:
!pip install -U spacy
import nltk
import spacy
from spacy.lang.en import English
!python -m spacy download en_core_web_md
!python -m spacy download en_core_web_sm-2.2.0 --direct
!pip install allennlp==1.0.0rc1 allennlp-models==1.0.0rc1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-05-09 04:44:50.978537: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 16.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
2023-05-09 04:45:05.604269: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  ERROR: HTTP error 404 while getting https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0-py3-none-any.whl
ERROR: Could not install requirement en-core-web-sm==2.2.0 from https://github.com/explosion/spacy-models/releases/d

In [4]:
from nltk.tokenize import word_tokenize
import pdb
import itertools
from itertools import permutations
from tqdm import tqdm
from nltk.corpus import stopwords
import string 
from difflib import SequenceMatcher
from spacy.pipeline import EntityRuler
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

True

In [5]:
from nltk.corpus import wordnet as wn
from nltk.tokenize.treebank import TreebankWordDetokenizer
import re
from queue import Queue 
from spacy.matcher import PhraseMatcher
import json
import csv

# READ FILE

In [6]:
from google.colab import files
uploaded = files.upload()

KeyboardInterrupt: ignored

In [7]:
for name, data in uploaded.items():
  filename = name

input_f = open(filename, 'r', encoding='utf-8')
content = input_f.read()
input_f.close()
output_filename = "output" + filename.split(".txt")[0] + ".json"

NameError: ignored

# Set Up all JOB Titles

In [8]:
#Fetching Job Titles from file and making a dictionary out of it
job_title_content=open("job_title_dataset.csv","r+")
job_title_data=csv.reader(job_title_content)
job_title=set()
flag=True;
for row in job_title_data:
  if flag:
    flag=False
    continue
  job_title.add(row[0].lower())
job_title_content.close()
job_title.discard("")

FileNotFoundError: ignored

In [9]:
def patternList(job_title):
  '''Storing Job titles dictionary in form of patterns List which is used for Entity Ruler of Spacy
    
    Args:
      job_title (dict): Dictionary of all the Job Titles

    Returns:
      final_pattern (list): List of all the Job Titles patterns generated 
  '''
  job_title.update(["board of directors", "chairman's advisory board", "buisness trading strategist" ])
  final_pattern = []
  for title in job_title:
    #removing garbage titles
    if len(title) <= 2:
      continue
    pattern = {"label": "TITLE", "pattern": ""}

    tokens = tokenizer(title)
    lower_list = []
    for title_sub in tokens:
      lower = {"LOWER": title_sub.text.lower()}
      lower_list.append(lower)
    pattern["pattern"] = lower_list

    final_pattern.append(pattern) 
  return final_pattern

# Part(Location, Location) Template

In [10]:
def getNERFeatures(doc):
  '''Retrieve all the Named Entity Recognition Features from sentence doc (spacy doc) and storing into dictionary
    
    Args:
      doc (spacy doc): doc obtained by passing spacy nlp pipeline 

    Returns:
      nerDict (dict): Named Entity Recognition Dictionary 
  ''' 

  nerDict = {}
  for x in doc.ents:
    nerDict[x.text]=x.label_
  return nerDict

In [11]:
def getWordnetFeatures(doc):
  '''Retrieve WordNet feature from sentence doc (spacy doc) and storing into dictionary
    
    Args:
      doc (spacy doc): doc obtained by passing spacy nlp pipeline 

    Returns:
      wordnetTokenDict (dict): WordNet Feature Dictionary
  ''' 
  wordnetTokenDict = {}
  tokens = []

  for ent in doc.ents:
    text = ent.text
    #convert two words sperate by '_' in order to get correct info from nltk wordnet
    #eg. San Francisco -> San_Francisco 
    text = text.replace(' ', '_')
    tokens.append(text)

  for token in tokens:
    tokenSynset = wn.synsets(token)
    synonyms=[]
    hypernymList=[]
    hyponymList=[]
    holonymList=[]
    meronymList=[]
    entailList=[]
    memberHolonymsList = []
  
    tempTokenDict={}
    for index, tempSynset in enumerate(tokenSynset):
      
      hypernyms = tempSynset.hypernyms()
      for l in hypernyms:
        hypernymList.append(l.lemma_names()[0])
        
      hyponyms = tempSynset.hyponyms()
      for l in hyponyms:
        hyponymList.append(l.lemma_names()[0])
        
      holonyms = tempSynset.part_holonyms()
      for l in holonyms:
        holonymList.append(l.lemma_names()[0])
        
      meronyms = tempSynset.part_meronyms()
      for l in meronyms:
        meronymList.append(l.lemma_names()[0])
        
      entail = tempSynset.entailments()
      for l in entail:
        entailList.append(l.lemma_names()[0])
        
      for l in tempSynset.lemmas():
        synonyms.append(l.name())
      
      memberHolonyms = tempSynset.member_holonyms()
      for l in memberHolonyms:
        memberHolonymsList.append(l.name())

      
    tempTokenDict["hypernym"]=hypernymList  
    tempTokenDict["hyponym"]=hyponymList
    tempTokenDict["holonym"]=holonymList
    tempTokenDict["meronym"]=meronymList
    tempTokenDict["entail"]=entailList
    tempTokenDict["synonym"]=synonyms
    tempTokenDict["member_holonyms"] = memberHolonymsList

    
    wordnetTokenDict[token.replace('_', ' ')]=tempTokenDict    
  
  return wordnetTokenDict

In [12]:
def partTemplate(sent, link_coref_sent, doc, EXTRACTION_LIST):
  '''Extracting Part(Location, Location) template from the sentence and storing
    all the extracted templates into EXTRACTION_LIST
    
    Args:
      sent (string): Sentence on which extraction is performed
      link_coref_sent (string): Orignal set of sentences from where correference
                                of current sentence is resolved.
      doc (spacy doc): doc obtained by passing spacy nlp pipeline
      EXTRACTION_LIST (list): Previously extracted templates list to which new templates will be appended
  ''' 
  #Fetching NER and WordNet Features
  nerDict = getNERFeatures(doc)
  wordnetFeatures = getWordnetFeatures(doc)

  part_of_template = set()
  possibleLocations = []
  for i in nerDict:
    if nerDict[i] == 'GPE' or nerDict[i] == 'LOC' or nerDict[i] == 'FAC' or nerDict[i] == 'NORP':
      possibleLocations.append(i)

  # If possible locations is less than 2 then we cannot find relation so return      
  if len(possibleLocations) < 2:
    return
  
  perm = list(permutations(possibleLocations, 2))
  
  #check direct relation by checking left's synonyms -> meronym of right
  for tuple in perm:
    left = tuple[0]
    right = tuple[1]

    leftSynonymSet = set([left.replace('', '_')])
    rightMeronymSet = set()

    leftSynonymSet.update(wordnetFeatures[left]["synonym"])
    rightMeronymSet.update(wordnetFeatures[right]["meronym"])

    intersectionSet = leftSynonymSet.intersection(rightMeronymSet)
    if len(intersectionSet) > 0:
      part_of_template.add(tuple)

  #check direct relation by checking left's holonym -> synonym of right
  for tuple in perm:
    left = tuple[0]
    right = tuple[1]

    leftHolonymSet = set()
    rightSynonymSet = set([right.replace(' ', '_')])

    leftHolonymSet.update(wordnetFeatures[left]["holonym"])
    rightSynonymSet.update(wordnetFeatures[right]["synonym"])

    intersectionSet = leftHolonymSet.intersection(rightSynonymSet)
    if len(intersectionSet) > 0:
      part_of_template.add(tuple)
  
  #check for transitive relations
  #i.e., left's holonym -> right's meronym
  for tuple in perm:
    left = tuple[0]
    right = tuple[1]

    leftHolonymSet = set()
    rightMeronymSet = set()

    leftHolonymSet.update(wordnetFeatures[left]["holonym"])
    rightMeronymSet.update(wordnetFeatures[right]["meronym"])

    intersectionSet = leftHolonymSet.intersection(rightMeronymSet)
    if len(intersectionSet) > 0:
      part_of_template.add(tuple)

  location_templates = []
  for tup in part_of_template:
    location_templates.append({"Location1": tup[0], "Location2": tup[1]})
  
  for temp in location_templates:
    final_part_template_format = {"template": "PART", 
                                "sentences": [link_coref_sent], 
                                "arguements": {"1": "",
                                          "2": ""
                                          }
                          } 
    final_part_template_format["arguements"]["1"] = temp["Location1"]
    final_part_template_format["arguements"]["2"] = temp["Location2"]
    EXTRACTION_LIST.append(final_part_template_format)

# BUY Template

In [13]:
def getLemma(verb):
  '''getting Lemma form of the input verb
    
    Args:
      verb (string): verb 

    Returns:
      lemmas (set): Lemmas of input verb 
  '''
  doc = nlp_other(verb)

  lemmas = set()
  for token in doc:
    lemmas.add(token.lemma_)
  
  return lemmas

In [14]:
def getSRLArguements(srl_obj):
  ''' Extract SRL Arguements and its value from SRL Object
      example the srl_obj["description"] = [ARG0: Amazon] [V: bought] [ARG1: Whole Food Market] [ARG3: for $ 1 Million Dollars] .
      so, we will extract the arguements and its value and store into arguement_map.
      Output: {"ARG0": "Amazon", "ARG1": "Whole Food Market", "ARG3": "for $ 1 Million Dollars"}
    
    Args:
      srl_obj (object): Semanctic Role Labelling Object obtained by applying 
                        AllenNlp SRL 

    Returns:
      arguement_map (dict): SRL Arguements extracted from srl_obj
  '''

  description = srl_obj["description"]
  arguements_box = re.findall("\[[^\]]*\]", description)

  arguement_map = {}

  for box in arguements_box:
    if len(box.split(":")) > 1:
      arguement, text = box.split(":")[0][1:], box.split(":")[1][:-1]

      if len(arguement) == 0 or len(text) == 0:
        continue
    arguement_map[arguement] = text.strip()
  
  return arguement_map


In [15]:
def buyTemplate(new_sent, link_coref_sent, srl_tags, doc1, EXTRACTION_LIST):
  ''' Extracting Buy(Buyer, Item, Price, Quantity, Source) template from the sentence and storing
    all the extracted templates into EXTRACTION_LIST
    
    Args:
      new_sent (string): Sentence on which extraction is performed
      link_coref_sent (string): Orignal set of sentences from where correference
                                of current sentence is resolved.
      srl_tags (object): SRL Tags obtained by performing AllenNlp SRL model on sentence
      doc1 (spacy doc): doc obtained by passing spacy nlp pipeline
      EXTRACTION_LIST (list): Previously extracted templates list to which new templates will be appended
  '''
  #set of buy and sell verbs which helps to trigger buying and selling event
  buy_verbs_set = set(["buy", "purchase", "get", "acquire", "capture", "take", "obtain", "retrive"])
  sell_verbs_set = set(["sell"])

  srl_verb_obj_list = srl_tags["verbs"]

  final_template_format = {"template": "BUY", 
                           "sentences": [link_coref_sent], 
                           "arguements": {"1": "",
                                          "2": "",
                                          "3": "",
                                          "4": "",
                                          "5": ""
                                          }
                          }   
  #spacy en_core_web_sm model
  doc2 = nlp_other(new_sent)

  buy_template = {"buyer": "", "item": "", "price": "", "quantity": "", "source": ""}
  for srl_verb_obj in srl_verb_obj_list:
    #get lemma form of verb to check with our buy_verbs_set and sell_verbs_set
    verb_lemmas = getLemma(srl_verb_obj["verb"])

    #Buy verb
    if len(buy_verbs_set.intersection(verb_lemmas)) > 0:

      arguement_map = getSRLArguements(srl_verb_obj)

      #running on sm and md model to fetch more results
      template1 = buy_md_test(new_sent, arguement_map, buy_template, doc1)
      template2 = buy_sm_test(new_sent, arguement_map, buy_template, doc2)
      buy_template = {"buyer": "", "item": "", "price": "", "quantity": "", "source": ""}

      for key in buy_template:
        if len(template1[key]) >= len(template2[key]):
          buy_template[key] = template1[key]
        else:
          buy_template[key] = template2[key]

      if buy_template["buyer"] != "" and buy_template["item"] != "":
        final_template_format["arguements"]["1"] = buy_template["buyer"]
        final_template_format["arguements"]["2"] = buy_template["item"]
        final_template_format["arguements"]["3"] = buy_template["price"]
        final_template_format["arguements"]["4"] = buy_template["quantity"]
        final_template_format["arguements"]["5"] = buy_template["source"]

        EXTRACTION_LIST.append(final_template_format)      

    
    #Sell verb
    if len(sell_verbs_set.intersection(verb_lemmas)) > 0:

      arguement_map = getSRLArguements(srl_verb_obj)

      #running on sm and md model to fetch more results
      template1 = sell_md_test(new_sent, arguement_map, buy_template, doc1)
      template2 = sell_sm_test(new_sent, arguement_map, buy_template, doc2)

      buy_template = {"buyer": "", "item": "", "price": "", "quantity": "", "source": ""}

      for key in buy_template:
        if len(template1[key]) >= len(template2[key]):
          buy_template[key] = template1[key]
        else:
          buy_template[key] = template2[key]

      if buy_template["buyer"] != "" and buy_template["item"] != "":
        final_template_format["arguements"]["1"] = buy_template["buyer"]
        final_template_format["arguements"]["2"] = buy_template["item"]
        final_template_format["arguements"]["3"] = buy_template["price"]
        final_template_format["arguements"]["4"] = buy_template["quantity"]
        final_template_format["arguements"]["5"] = buy_template["source"]

        EXTRACTION_LIST.append(final_template_format)      

In [16]:
def buy_sm_test(new_sent, arguement_map, buy_template, doc):
  ''' Extracting Buy(Buyer, Item, Price, Quantity, Source) template from the sentence 
      by running spacy's sm model.
      This is called when Buying event is triggered.
      
    Args:
      new_sent (string): Sentence on which extraction is performed
      arguement_map (dict): SRL obj arguement dictionary
      buy_template (dict): buy template blank format which needs to be filled
      doc (spacy doc): doc obtained by passing spacy nlp pipeline
    
    Returns:
      buy_template (dict): BUY template extracted
  '''
  buy_template = {"buyer": "", "item": "", "price": "", "quantity": "", "source": ""}
  
  if "ARG0" in arguement_map:
        
    #extracting BUYER from ARG0 and SELLER from ARG2
    for ent in doc.ents:
      if ent.label_ in ["PERSON", "NORP", "GPE", "ORG"] and ent.text in arguement_map["ARG0"]:
        buy_template["buyer"] = ent.text
      elif ent.label_ in ["PERSON", "NORP", "GPE", "ORG"] and "ARG2" in arguement_map and ent.text in arguement_map["ARG2"]:
        buy_template["source"] = ent.text
    
    
    #extracting ITEM bought from ARG1
    item_list=[]
    for ent in doc.ents:
      # print(ent)
      if ent.label_ in ["PERSON", "NORP", "ORG", "PRODUCT"] and "ARG1" in arguement_map and ent.text in arguement_map["ARG1"]:
        item_list.append(ent.text)
    buy_template["item"] = ";".join(item_list)
    
    #extracting MONEY from ARG3 if available
    for ent in doc.ents:
      
      ent_tokenize = set(word_tokenize(ent.text))

      if ent.label_ == "MONEY":
        if "ARG3" in arguement_map:
          arg3_tokenize = set(word_tokenize(arguement_map["ARG3"]))
          if len(ent_tokenize.intersection(arg3_tokenize)) == len(ent_tokenize):
            buy_template["price"] = ent.text
        else:
          arg1_tokenize = set(word_tokenize(arguement_map["ARG1"]))
          if len(ent_tokenize.intersection(arg1_tokenize)) == len(ent_tokenize):
            buy_template["price"] = ent.text
    
    #extracting QUANTITY if present by checking NER of Quantity or Cardinal
    for ent in doc.ents:
      quantity_ner = []
      cardinal_ner = []
      for ent in doc.ents:
        if ent.label_ == "QUANTITY":
          quantity_ner.append(ent.text)
        elif ent.label_ == "CARDINAL":
          cardinal_ner.append(ent.text)
      
      if len(quantity_ner) > 0:
        buy_template["quantity"] = ";".join(quantity_ner)
      else:
        buy_template["quantity"] = ";".join(cardinal_ner)
  return buy_template
        

In [17]:
def buy_md_test(new_sent, arguement_map, buy_template, doc):
  ''' Extracting Buy(Buyer, Item, Price, Quantity, Source) template from the sentence 
      by running spacy's md model.
      This is called when Buying event is triggered.
      
    Args:
      new_sent (string): Sentence on which extraction is performed
      arguement_map (dict): SRL obj arguement dictionary
      buy_template (dict): buy template blank format which needs to be filled
      doc (spacy doc): doc obtained by passing spacy nlp pipeline
    
    Returns:
      buy_template (dict): BUY template extracted
  '''
  buy_template = {"buyer": "", "item": "", "price": "", "quantity": "", "source": ""}
  
  if "ARG0" in arguement_map:
        
    #extracting BUYER from ARG0 and SELLER from ARG2
    for ent in doc.ents:
      if ent.label_ in ["PERSON", "NORP", "GPE", "ORG"] and ent.text in arguement_map["ARG0"]:
        buy_template["buyer"] = ent.text
      elif ent.label_ in ["PERSON", "NORP", "GPE", "ORG"] and "ARG2" in arguement_map and ent.text in arguement_map["ARG2"]:
        buy_template["source"] = ent.text
    
    
    #extracting ITEM bought from ARG1
    item_list=[]
    for ent in doc.ents:
      if ent.label_ in ["PERSON", "NORP", "ORG", "PRODUCT"] and "ARG1" in arguement_map and ent.text in arguement_map["ARG1"]:
        item_list.append(ent.text)
    buy_template["item"] = ";".join(item_list)
    
    #extracting MONEY from ARG3 if available
    for ent in doc.ents:      
      ent_tokenize = set(word_tokenize(ent.text))

      if ent.label_ == "MONEY":
        if "ARG3" in arguement_map:
          arg3_tokenize = set(word_tokenize(arguement_map["ARG3"]))
          if len(ent_tokenize.intersection(arg3_tokenize)) == len(ent_tokenize):
            buy_template["price"] = ent.text
        else:
          arg1_tokenize = set(word_tokenize(arguement_map["ARG1"]))
          if len(ent_tokenize.intersection(arg1_tokenize)) == len(ent_tokenize):
            buy_template["price"] = ent.text
    
    #extracting QUANTITY if present by checking NER of Quantity or Cardinal
    for ent in doc.ents:
      quantity_ner = []
      cardinal_ner = []
      for ent in doc.ents:
        if ent.label_ == "QUANTITY":
          quantity_ner.append(ent.text)
        elif ent.label_ == "CARDINAL":
          cardinal_ner.append(ent.text)
      
      if len(quantity_ner) > 0:
        buy_template["quantity"] = ";".join(quantity_ner)
      else:
        buy_template["quantity"] = ";".join(cardinal_ner)
  return buy_template
        

In [18]:
def sell_sm_test(new_sent, arguement_map, buy_template, doc):
  ''' Extracting Buy(Buyer, Item, Price, Quantity, Source) template from the sentence 
      by running spacy's sm model.
      This is called when selling event is triggered.
    
    Args:
      new_sent (string): Sentence on which extraction is performed
      arguement_map (dict): SRL obj arguement dictionary
      buy_template (dict): buy template blank format which needs to be filled
      doc (spacy doc): doc obtained by passing spacy nlp pipeline
    
    Returns:
      buy_template (dict): BUY template extracted
  '''
  buy_template = {"buyer": "", "item": "", "price": "", "quantity": "", "source": ""}
  
  if "ARG2" in arguement_map:
        
    #extracting BUYER from ARG2 and SELLER from ARG0
    for ent in doc.ents:
      if ent.label_ in ["PERSON", "NORP", "GPE", "ORG"] and ent.text in arguement_map["ARG2"]:
        buy_template["buyer"] = ent.text
      elif ent.label_ in ["PERSON", "NORP", "GPE", "ORG"] and "ARG0" in arguement_map and ent.text in arguement_map["ARG0"]:
        buy_template["source"] = ent.text
    
    
    #extracting ITEM sold from ARG1
    item_list=[]
    for ent in doc.ents:
      if ent.label_ in ["PERSON", "NORP", "ORG", "PRODUCT"] and "ARG1" in arguement_map and ent.text in arguement_map["ARG1"]:
        item_list.append(ent.text)
    buy_template["item"] = ";".join(item_list)
    
    #extracting MONEY from ARG3 if available
    for ent in doc.ents:
      ent_tokenize = set(word_tokenize(ent.text))

      if ent.label_ == "MONEY":
        if "ARG3" in arguement_map:
          arg3_tokenize = set(word_tokenize(arguement_map["ARG3"]))
          if len(ent_tokenize.intersection(arg3_tokenize)) == len(ent_tokenize):
            buy_template["price"] = ent.text
        else:
          arg1_tokenize = set(word_tokenize(arguement_map["ARG1"]))
          if len(ent_tokenize.intersection(arg1_tokenize)) == len(ent_tokenize):
            buy_template["price"] = ent.text
    
    #extracting QUANTITY if present by checking NER of Quantity or Cardinal
    for ent in doc.ents:
      quantity_ner = []
      cardinal_ner = []
      for ent in doc.ents:
        if ent.label_ == "QUANTITY":
          quantity_ner.append(ent.text)
        elif ent.label_ == "CARDINAL":
          cardinal_ner.append(ent.text)
      
      if len(quantity_ner) > 0:
        buy_template["quantity"] = ";".join(quantity_ner)
      else:
        buy_template["quantity"] = ";".join(cardinal_ner)
  return buy_template

In [19]:
def sell_md_test(new_sent, arguement_map, buy_template, doc):
  ''' Extracting Buy(Buyer, Item, Price, Quantity, Source) template from the sentence 
      by running spacy's md model.
      This is called when selling event is triggered.
    
    Args:
      new_sent (string): Sentence on which extraction is performed
      arguement_map (dict): SRL obj arguement dictionary
      buy_template (dict): buy template blank format which needs to be filled
      doc (spacy doc): doc obtained by passing spacy nlp pipeline
    
    Returns:
      buy_template (dict): BUY template extracted
  '''
  buy_template = {"buyer": "", "item": "", "price": "", "quantity": "", "source": ""}
  
  if "ARG2" in arguement_map:
        
    #extracting BUYER from ARG2 and SELLER from ARG0
    for ent in doc.ents:
      if ent.label_ in ["PERSON", "NORP", "GPE", "ORG"] and ent.text in arguement_map["ARG2"]:
        buy_template["buyer"] = ent.text
      elif ent.label_ in ["PERSON", "NORP", "GPE", "ORG"] and "ARG0" in arguement_map and ent.text in arguement_map["ARG0"]:
        buy_template["source"] = ent.text
    
    
    #extracting ITEM sold from ARG1
    item_list=[]
    for ent in doc.ents:
      if ent.label_ in ["PERSON", "NORP", "ORG", "PRODUCT"] and "ARG1" in arguement_map and ent.text in arguement_map["ARG1"]:
        item_list.append(ent.text)
    buy_template["item"] = ";".join(item_list)
    
    #extracting MONEY from ARG3 if available
    for ent in doc.ents:
      ent_tokenize = set(word_tokenize(ent.text))

      if ent.label_ == "MONEY":
        if "ARG3" in arguement_map:
          arg3_tokenize = set(word_tokenize(arguement_map["ARG3"]))
          if len(ent_tokenize.intersection(arg3_tokenize)) == len(ent_tokenize):
            buy_template["price"] = ent.text
        else:
          arg1_tokenize = set(word_tokenize(arguement_map["ARG1"]))
          if len(ent_tokenize.intersection(arg1_tokenize)) == len(ent_tokenize):
            buy_template["price"] = ent.text
    
    #extracting QUANTITY if present by checking NER of Quantity or Cardinal
    for ent in doc.ents:
      quantity_ner = []
      cardinal_ner = []
      for ent in doc.ents:
        if ent.label_ == "QUANTITY":
          quantity_ner.append(ent.text)
        elif ent.label_ == "CARDINAL":
          cardinal_ner.append(ent.text)
      
      if len(quantity_ner) > 0:
        buy_template["quantity"] = ";".join(quantity_ner)
      else:
        buy_template["quantity"] = ";".join(cardinal_ner)
  return buy_template

# Work Template

In [20]:
#list of all the stopwords
stopwordSet=set(stopwords.words('english'))

In [21]:
def checkWorkTrigger(new_sent, doc):
  '''Check if the given sentence has any work template based on entities present 
    in the sentence. If Person, Organization and Job Title entity is present then 
    there's chance that it might have work template present so we will return all
    entities related to Person, Organization and Job Title.
    
    Args:
      new_sent (string): Sentence on which extraction is performed
      doc (spacy doc): doc obtained by passing spacy nlp pipeline

    Returns:
      person_entities (list): list of Person entities present in the sentence
      title_entities (list): list of Job Title entities present in the sentence
      organization_entities (list): list of organization entities present in the sentence
      trigged: flag saying if the work template is triggered or not based on entities present
  '''
  person_entities = set()
  organization_entities = set()
  title_entities = set()
  for ent in doc.ents:
    if ent.label_ == "PERSON":
      person_entities.add(ent.text)
    elif ent.label_ == "ORG":
      organization_entities.add(ent.text)
    elif ent.label_ == "TITLE":
      title_entities.add(ent.text)
  if len(person_entities) > 0 or len(organization_entities) > 0 or len(title_entities) > 0:
    trigged = True
  else:
    trigged = False
  return person_entities, title_entities, organization_entities, trigged

In [22]:
def workTemplate(new_sent, link_coref_sent, srl_tags, doc1, EXTRACTION_LIST):
  ''' Extracting WORK(Person, Organization, Position, Location) template from the sentence and storing
    all the extracted templates into EXTRACTION_LIST
    
    Args:
      new_sent (string): Sentence on which extraction is performed
      link_coref_sent (string): Orignal set of sentences from where correference
                                of current sentence is resolved.
      srl_tags (object): SRL Tags obtained by performing AllenNlp SRL model on sentence
      doc1 (spacy doc): doc obtained by passing spacy nlp pipeline
      EXTRACTION_LIST (list): Previously extracted templates list to which new templates will be appended
  '''
  final_work_template_format = {"template": "WORK", 
                                "sentences": [link_coref_sent], 
                                "arguements": {"1": "",
                                          "2": "",
                                          "3": "",
                                          "4": ""
                                          }
                              }
  #check if the sentence has possiblity of work template
  person_entities, title_entities, organization_entities, trigged = checkWorkTrigger(new_sent, doc1)

  if not trigged:
    return
  
  srl_verb_obj_list = srl_tags["verbs"]
  templates_extracted = {}
  
  #SRL verb "became" eg: Steve Jobs 'became' CEO of Apple.
  work_SRL_verb_became(person_entities, title_entities, organization_entities, srl_verb_obj_list, templates_extracted)

  #SRL verb "worked" eg: Steve Jobs 'worked' as CEO of Apple.
  work_SRL_verb_worked(person_entities, title_entities, organization_entities, srl_verb_obj_list, templates_extracted)

  #Dependency parser
  work_dependency_parsing(doc1, person_entities, templates_extracted)
  
  #Phrase Matcher
  work_phrase_matcher(doc1, templates_extracted)

  for temp in templates_extracted.values():
    final_work_template_format = {"template": "WORK", 
                                "sentences": [link_coref_sent], 
                                "arguements": {"1": "",
                                          "2": "",
                                          "3": "",
                                          "4": ""
                                          }
                              }
    
    final_work_template_format["arguements"]["1"] = temp["Person"]
    final_work_template_format["arguements"]["2"] = temp["Organization"]
    final_work_template_format["arguements"]["3"] = ";".join(temp["Position"])  #concatenate if person has multiple position at same org
    final_work_template_format["arguements"]["4"] = temp["Location"]

    EXTRACTION_LIST.append(final_work_template_format)


In [23]:
def work_SRL_verb_became(person_entities, title_entities, organization_entities, srl_verb_obj_list, templates_extracted):
  '''check if the ROOT verb is 'became' and the AGENT of the verb is Person, THEME is Organization 
    
    Args:
      person_entities (list): list of person entities
      title_entities (list): list of title entities
      organization_entities (list): list of organization entities
      srl_verb_obj_list (list): srl verb object list
      templates_extracted (list): previously extracted templates for WORK Template for given sentence
  '''
  for srl_verb_obj in srl_verb_obj_list:
    #if the verb is 'became'
    if srl_verb_obj["verb"] == "became":
        arguement_map = getSRLArguements(srl_verb_obj)
        
        if "ARG1" in arguement_map and "ARG2" in arguement_map:

          work_template = {"Person": "", "Organization": "", "Position": "", "Location": ""}

          #Looking for Person entity
          for person in person_entities:
            if person in arguement_map["ARG1"]:
              work_template["Person"] = person

          position_set = set()

          #Looking for Title Entity
          for title in title_entities:
            if title in arguement_map["ARG2"]:
              position_set.add(title)
          
          if len(position_set) != 0:
            work_template["Position"] = position_set

          #Looking for Organization Entity
          for organization in organization_entities:
            if organization in arguement_map["ARG2"]:
              work_template["Organization"] = organization
          
          

          if work_template["Person"] != "" and work_template["Organization"] != "":
            hash = work_template["Person"] + "#" + work_template["Organization"]
            if hash in template_extracted:
              obj = template_extracted[hash]
              obj["Position"].update(work_template["Position"])
            else:
              template_extracted[hash] = work_template

In [24]:
def work_SRL_verb_worked(person_entities, title_entities, organization_entities, srl_verb_obj_list, template_extracted):
  '''check if the ROOT verb is 'worked' and the AGENT of the verb is Person, THEME is Organization 
    
    Args:
      person_entities (list): list of person entities
      title_entities (list): list of title entities
      organization_entities (list): list of organization entities
      srl_verb_obj_list (list): srl verb object list
      templates_extracted (list): previously extracted templates for WORK Template for given sentence
  '''
  for srl_verb_obj in srl_verb_obj_list:
    if srl_verb_obj["verb"] == "worked":
      arguement_map = getSRLArguements(srl_verb_obj)
      
      #check only if AGENT is present that is Person entity working
      if "ARG0" in arguement_map:
        work_template = {"Person": "", "Organization": "", "Position": "", "Location": ""}

        #Looking for Person Entity
        for person in person_entities:
          if person in arguement_map["ARG0"]:
            work_template["Person"] = person

        position_set = set()

        #Looking for Title Entity
        for title in title_entities:
          if ("ARG1" in arguement_map and title in arguement_map["ARG1"]) or ("ARG2" in arguement_map and title in arguement_map["ARG2"]):
            position_set.add(title)
        
        if len(position_set) != 0:
          work_template["Position"] = position_set

        #Looking for Organization Entity
        for organization in organization_entities:
          if ("ARG1" in arguement_map and organization in arguement_map["ARG1"]) or ("ARG2" in arguement_map and organization in arguement_map["ARG2"]):
            work_template["Organization"] = organization
        

        if work_template["Person"] != "" and work_template["Organization"] != "":
          hash = work_template["Person"] + "#" + work_template["Organization"]
          if hash in template_extracted:
            obj = template_extracted[hash]
            obj["Position"].update(work_template["Position"])
          else:
            template_extracted[hash] = work_template

In [25]:
def work_phrase_matcher(doc, template_extracted):
  '''check if the sentence matches particular phrase like 
    'Apple co-founder Steve Jobs',
    'Jeff Bezos, CEO of Amazon'
    This uses spacy phrase matcher pipeline
    
    Args:
      doc (spacy doc): doc obtained by passing spacy nlp pipeline
      templates_extracted (list): previously extracted templates for WORK Template for given sentence
  '''
  
  for match_id, start, end in matcher(doc):
    dict_work={"Person": "", "Organization": "", "Position": "", "Location": ""}
    for ent in doc[start:end].ents:
      if ent.label_ == "PERSON":
        dict_work["Person"]=ent.text
      if ent.label_ == "ORG":
        dict_work["Organization"]=ent.text
      if ent.label_ == "TITLE":
        dict_work["Position"]=ent
    
    hash = dict_work["Person"] + "#" + dict_work["Organization"]
    if hash in template_extracted:
      obj = template_extracted[hash]
      obj["Position"].add(dict_work["Position"])
    else:
      template_extracted[hash] = dict_work

In [26]:
def work_dependency_parsing(doc, person_entities, template_extracted):
  '''Applying dependency parsing technique, extracting the work template. Here we use
    organization whose dependency parsing is pobj and from there we are tracing back to
    positions  as well as the person entity using dependency parser.
    
    Args:
      doc (spacy doc): doc obtained by passing spacy nlp pipeline
      person_entities (list): list if person entity for current sentence
      templates_extracted (list): previously extracted templates for WORK Template for given sentence
  '''
  company_index1 = 0
  for token in doc:
    template = {"Person": "", "Position": "", "Organization": "", "Location" : ""}
    if token.dep_ == "pobj" and token.ent_type_ in ["PERSON","ORG","GPE","NORP"] and token.head.dep_ == "prep" and token.head.head.ent_type_ == "TITLE":
      
      template["Organization"] = token.text
      current_company_index = token.i
      position1 = token.head.head
      positions_set = set([position1.text])

      #using queue to find other conj and appos related to the position1
      q = Queue()
      for child in position1.conjuncts:
        q.put(child)
      
      while not q.empty():
        position = q.get()
        if  position.ent_type_ == "TITLE" and position.i >= company_index1 and position.i <= current_company_index:
          positions_set.add(position.text)
          for child in position.children:
            if child.dep_ == "appos":
              q.put(child)
      company_index1 = current_company_index
      template["Position"] = positions_set

      if len(person_entities) == 1:
        template["Person"] = next(iter(person_entities))
      else:
        pflag=False
        for ancestor in token.ancestors:
          if ancestor.pos_ == "AUX" and not pflag:
            for child in ancestor.children:
              if child.dep_ == "nsubj":
                template["Person"] = child.text
                pflag=True
                break        
    else:
      if token.dep_ == "pobj" and token.ent_type_ in ["PERSON","ORG","GPE","NORP"] and token.head.dep_ == "prep" and token.head.head.pos_ == "VERB":
        template = {"Person": "", "Position": "", "Organization": "", "Location" : ""}

        template["Organization"] = token.text

        current_company_index = token.i
        verb = token.head.head

        for verb_child in verb.children:
          if verb_child.dep_ == "prep":
            for verb_grand_child in verb_child.children:
              if verb_grand_child.dep_ == "pobj" and verb_grand_child.ent_type_ == "TITLE":
                position1 = verb_grand_child
                positions_set = set([position1.text])

                q = Queue()
                for child in position1.conjuncts:
                  q.put(child)
        
                while not q.empty():
                  position = q.get()
                  if  position.ent_type_ == "TITLE" and position.i >= company_index1 and position.i <= current_company_index:
                    positions_set.add(position.text)

                    for child in position.children:
                      if child.dep_ == "appos":
                        q.put(child)
                template["Position"] = positions_set

                if len(person_entities) == 1:
                  template["Person"] = next(iter(person_entities))
                else:
                  pflag=False
                  for ancestor in token.ancestors:
                    if ancestor.pos_ == "AUX" and not pflag:
                      for child in ancestor.children:
                        if child.dep_ == "nsubj":
                          template["Person"] = child.text
                          pflag=True
                          break        
        company_index1 = current_company_index

    if template["Person"] != "" and template["Organization"] != "":
      hash = template["Person"] + "#" + template["Organization"]
      if hash in template_extracted:
        obj = template_extracted[hash]
        obj["Position"].update(template["Position"])
      else:
        template_extracted[hash] = template

# Correference Resolution and  Template Extraction

In [28]:
import spacy
import en_core_web_sm, en_core_web_md

#Over here we will use en_core_web_sm model for all our task but some task which
#requires increasing accuracy of the extraction we will use en_Core_web_md model
nlp_other = en_core_web_md.load()
nlp_all = en_core_web_sm.load()


In [30]:
#adding sentencizer to nlp pipeline to segment paragraphs into sentences
sent_pipe = nlp_other.create_pipe("sentencizer")
nlp_other.add_pipe("sentencizer")

#removing unnecessary pipes
remove_pipes_other = [p for p in nlp_other.pipe_names if p not in ["tagger", "sentencizer", "ner"]]

In [36]:
#Entity Ruler for Job Title which will eventually added to spacy pipeline
ruler = EntityRuler(nlp_all)




#disabling pipes for corref
nlp_other.disable_pipes(remove_pipes_other)

['tok2vec', 'parser', 'attribute_ruler', 'lemmatizer']

In [38]:
#Work Phrase Matcher Initialization
matcher = PhraseMatcher(nlp_all.vocab, attr="ENT_TYPE")
matcher.add("Work_Temp", None, nlp_all("Amazon's co-founder Jeff Bezos"), nlp_all("Apple CEO Steve Jobs"))

100%|██████████| 1345947288/1345947288 [00:21<00:00, 64026329.32B/s]


Did not use initialization regex that was passed: _context_layer._module.weight_hh.*
Did not use initialization regex that was passed: _context_layer._module.weight_ih.*
100%|██████████| 406056588/406056588 [00:06<00:00, 61091905.03B/s]


In [39]:
def checkNER(tokens, start_index, end_index, ent_para):
  '''check whether any entity is present in token whose start index and end index 
    is given
    
    Args:
      tokens (list): sentence token list
      start_index (int): start index
      end_index (int): end index
      ent_para (list): list of entities in the given sentence

    Returns:
      flag (boolean): returns True if entity present, False otherwise
  '''
  aggregate = " ".join(tokens[start_index: end_index])
  if aggregate.strip() in ent_para:
    return True
  return False

In [40]:
def resolveClustersFromCoref(clusters, ent_para):
  '''Intermediate step for resolving correference, 
    for each clusters it check whether it needs to replace the token with any entity,
    based on that cluster_resolve_map is created which helps in resolving correference
    at later stage

    Args:
      clusters (list of list): clusters formed by using allennlp correference resolution
      ent_para (list): entities present in current paragraph

    Returns:
      cluster_resolve_map (dict): map for resolving cluster
  '''
  cluster_resolve_map = {}
  for cluster in clusters:
    orignal_start_index = cluster[0][0]
    orignal_end_index = cluster[0][1] + 1
    for i in range(1, len(cluster)):
      start_index = cluster[i][0]
      end_index = cluster[i][1] + 1
      if checkNER(tokens, start_index, end_index, ent_para):
        orignal_start_index = start_index
        orignal_end_index = end_index
      #map of :
      #start_index_to_resolve -> ([start_index_to_resolve: end_index_to_resolve], resolved word, resolved word index)        
      cluster_resolve_map[start_index] = ([start_index, end_index], " ".join(tokens[orignal_start_index: orignal_end_index]), orignal_start_index)
  return cluster_resolve_map

In [41]:
def processSentence(orignalSent, sentDocs, sent_index, start_index, cluster_resolve_map, token_index_map, EXTRACTION_LIST):
  '''This function is for processing the sentence. First, correference resolution is performed
    which results in new sentence with resolved pronouns. Once correfenced sentence is obtained
    we try to find templates from the sentence (PART, BUY and WORK Template)

    Args:
      orignalSent (string): Orignal sentence from the wikipedia document
      sentDocs (spacy doc): doc obtained when ran on the sentence
      sent_index (int): index of the current sentence in the paragraph
      start_index (int): start index of the first token of current sentence relative to paragraph
      cluster_resolve_map (dict): map for resolving clusters obtained by using allennlp coref
      token_index_map (dict): map where each token index is related to corresponding sentence index
      EXTRACTION_LIST (list): Previously extracted templates list to which new templates will be appended

    Returns:
      len (int): total tokens in given sentence
  '''
  newSent = []
  backTrackSentIndex = sent_index

  orignalTokens = []
  for token in orignalSent:
    orignalTokens.append(token.text)

  #storing token_number -> sent_index
  for i in range(0, len(orignalTokens)):
    token_index_map[start_index + i] = sent_index
  
  i = 0
  while(i < len(orignalTokens)):
    #check if it needs resolution
    if (start_index + i) in cluster_resolve_map:
      tup = cluster_resolve_map[start_index + i]

      #range to resolve
      start = tup[0][0] - start_index
      end = tup[0][1] - start_index
      newSent.append(tup[1])
      

      #find resolved sent index
      if tup[1] != orignalTokens[i]:
        resolving_sent_index = token_index_map[tup[2]]
        if resolving_sent_index < backTrackSentIndex:
          backTrackSentIndex = resolving_sent_index
      
      i = end
    else:
      newSent.append(orignalTokens[i])
      i = i + 1
  link_coref_sent = ""
  iter = itertools.islice(sentDocs.sents, backTrackSentIndex, sent_index + 1)
  for iteration in iter:
    link_coref_sent += " " + iteration.text

  #replacing words like 'co - founder' -> 'co-founder'
  new_sent =  TreebankWordDetokenizer().detokenize(newSent).replace(" - ", "-")

  #srl tags
  srl_tags = predictor_srl.predict(sentence=new_sent)

  #Applying spacy pipeline to correferenced sentence obtained
  doc1 = nlp_all(new_sent)

  # BUY TEMPLATE
  buyTemplate(new_sent, link_coref_sent, srl_tags, doc1, EXTRACTION_LIST)

  # WORK TEMPLATE
  workTemplate(new_sent, link_coref_sent, srl_tags, doc1, EXTRACTION_LIST)

  #LOCATION TEMPLATE
  partTemplate(new_sent, link_coref_sent, doc1, EXTRACTION_LIST)
  return len(orignalTokens)


In [42]:
#spliting wikipedia documents by paragraphs
paras = content.split("\n")

NameError: ignored

# Task1(Get Features)


In [44]:
from nltk.corpus import wordnet
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [45]:
def get_wn_feature(tokenSynset):
  '''getting wordnet features from tokensynset

    Args:
      tokenSynset (list): list of token synsets 

    Returns:
      synonyms
      hypernymList
      hyponymList
      holonymList
      meronymList
      entailList
  '''
  synonyms=[]
  hypernymList=[]
  hyponymList=[]
  holonymList=[]
  meronymList=[]
  entailList=[]
  memberHolonymsList = []
  for index, tempSynset in enumerate(tokenSynset):
      
      hypernyms = tempSynset.hypernyms()
      for l in hypernyms:
        hypernymList.append(l.lemma_names()[0])
        
      hyponyms = tempSynset.hyponyms()
      for l in hyponyms:
        hyponymList.append(l.lemma_names()[0])
        
      holonyms = tempSynset.part_holonyms()
      for l in holonyms:
        holonymList.append(l.lemma_names()[0])
        
      meronyms = tempSynset.part_meronyms()
      for l in meronyms:
        meronymList.append(l.lemma_names()[0])
        
      entail = tempSynset.entailments()
      for l in entail:
        entailList.append(l.lemma_names()[0])
        
      for l in tempSynset.lemmas():
        synonyms.append(l.name())
      
      memberHolonyms = tempSynset.member_holonyms()
      for l in memberHolonyms:
        memberHolonymsList.append(l.name())
  return synonyms,hypernymList,hyponymList,holonymList,meronymList,entailList

In [46]:
def get_features_sent(doc):
  '''Extracting features like wordnet features, pos tags, dependency, entity type
    from the spacy doc of the sentence.
    
    Args:
      doc (spacy doc): doc obtained from spacy pipeline on sentence
    
    Returns:
      featureList (list): list of features for the sentence
  '''
  featureList={}
  for token in doc:
    if token.dep_ != 'punct' and token.dep_!='':
      token_wn=token.text.replace(" ","_")
      featureDict = {}
      t_synset=wordnet.synsets(token_wn)
      synonyms_List,hypernym_List,hyponym_List,holonym_List,meronym_List,entail_List = get_wn_feature(t_synset)
      featureDict["synonyms"]=synonyms_List
      featureDict["hypernyms"]=hypernym_List
      featureDict["hyponyms"]=hyponym_List
      featureDict["holonyms"]=holonym_List
      featureDict["meronyms"]=meronym_List
      featureDict["entailments"]=entail_List
      featureDict["token"] = token
      featureDict["lemmaVal"] = token.lemma
      featureDict["lemma"] = token.lemma_
      featureDict["pos"] = token.pos_
      featureDict["tag"] = token.tag_
      featureDict["shape"] = token.shape_
      featureDict["isAlpha"] = token.is_alpha
      featureDict["isStop"] = token.is_stop
      featureDict["dependency"]=token.dep_
      featureDict["headText"] = token.head.text
      featureDict["headTag"]=token.head.pos_
      featureDict["entity_rel"]=token.ent_type_
      featureDict["ent_type"]=token.ent_id
      featureDict["children"] = [child for child in token.children]
      featureList[token.text]=featureDict
  return featureList

In [47]:
doc=nlp_all("Powell Jobs is a founding member of the Climate Leadership Council.")
print(get_features_sent(doc))

{'Powell': {'synonyms': ['Powell', 'Colin_Powell', 'Colin_luther_Powell', 'Powell', 'Cecil_Frank_Powell'], 'hypernyms': [], 'hyponyms': [], 'holonyms': [], 'meronyms': [], 'entailments': [], 'token': Powell, 'lemmaVal': 7187880078887531347, 'lemma': 'Powell', 'pos': 'PROPN', 'tag': 'NNP', 'shape': 'Xxxxx', 'isAlpha': True, 'isStop': False, 'dependency': 'compound', 'headText': 'Jobs', 'headTag': 'PROPN', 'entity_rel': 'PERSON', 'ent_type': 0, 'children': []}, 'Jobs': {'synonyms': ['occupation', 'business', 'job', 'line_of_work', 'line', 'job', 'task', 'chore', 'job', 'job', 'job', 'job', 'job', 'problem', 'job', 'Job', 'Job', 'job', 'Job', 'Book_of_Job', 'caper', 'job', 'job', 'subcontract', 'farm_out', 'job', 'job', 'speculate', 'job'], 'hypernyms': ['activity', 'duty', 'workplace', 'product', 'duty', 'work', 'work', 'difficulty', 'unfortunate', 'application', 'robbery', 'cheat', 'hire', 'work', 'invest'], 'hyponyms': ['accountancy', 'appointment', 'career', 'catering', 'confectionery

# ROUGH WORK

In [ ]:
gpe = [] # countries, cities, states
loc = [] # non gpe locations, mountain ranges, bodies of water


doc = nlp("Richardson is principal city in Dallas which is in United States.")
for ent in doc.ents:
    if (ent.label_ == 'GPE'):
        gpe.append(ent.text)
    elif (ent.label_ == 'LOC'):
        loc.append(ent.text)

print(gpe)
print(loc)
cities = []
countries = []
other_places = []
import wikipedia
for text in gpe:
    summary = str(wikipedia.summary(text))
    # print(summary)
    if ('city' in summary):
        cities.append(text)
    elif ('country' in summary):
        countries.append(text)
    else:
        other_places.append(text)

for text in loc:
    other_places.append(text)

In [ ]:
cities

In [ ]:
countries

In [ ]:
!pip install pycountry

In [ ]:
import pycountry
text = "U.S (New York), United Kingdom (London)"
for country in pycountry.countries:
    if country.name in text:
        print(country.name)

In [ ]:
!pip install geotext
from geotext import GeoText

In [ ]:
gpe = [] # countries, cities, states
loc = [] # non gpe locations, mountain ranges, bodies of water


doc = nlp("Richardson is principal city in Dallas which is in United States.")
for ent in doc.ents:
    if (ent.label_ == 'GPE'):
        gpe.append(ent.text)
    elif (ent.label_ == 'LOC'):
        loc.append(ent.text)

In [ ]:
places = GeoText("Richardson is principal city in Dallas which is in U.S.")

In [ ]:

cities = list(places.cities)
cities


In [ ]:
places.countries

In [ ]:
places.nationalities